# Leitura do CSV (Bronze)

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("inep").getOrCreate()

df_bronze = spark.read.option("header",True).option("encoding","utf-8").csv("escolas_inep.csv")
df_bronze.printSchema()
df_bronze.show(5)

df_bronze.write.mode("overwrite").parquet("data/processed/bronze_escolas.parquet")
